### import 및 파일읽기
- 일반적인 datadisk 양식에 맞추어 자산정리 내용을 출력할 수 있도록 하기 위한 코드
- datadisk 무담보 기준으로 작성되었음. 조정채권/실효폐지채권은 추가 작업 필요

In [1]:
import pandas as pd
import numpy as np
import pickle, re, os
from os.path import join
import warnings
warnings.simplefilter("ignore")
import functions

pd.options.display.float_format = '{:,.0f}'.format
dtype = functions.dtype
path_dataset = functions.path_dataset

In [35]:
#################
company = "솔림"
basedate = "250430"
fn_keys = "키리스트.xlsx"
#################
path_excel = join(path_dataset, company, basedate)
path_pkl = join(path_excel, "pkl")

# 파일검색어
파일검색어 = ["채무자조회새창_", "계좌조회새창_", "보증인새창_.*새보증인상태", "법조치조회새창", "전체_원데이터"]
        
# 파일 읽기
    # 기본 파일
if True :
    # 파일명리스트
    files = []        
    for i in range(len(파일검색어)) : 
        if re.search("보증인새창", 파일검색어[i]) : # 새보증인상태 파일 말고 원 파일 읽자
            files.append(functions.키워드로파일명찾기(path_pkl, 파일검색어[i], 전체경로=True))
        else : 
            files.append(functions.키워드로파일명찾기(path_pkl, 파일검색어[i],전체경로=True))
                    
    # 파일 읽기
        # 전산
    debt = pd.read_pickle(files[0])
    account = pd.read_pickle(files[1])
    grt = pd.read_pickle(files[2])   # 종결, 종료에 해당하는 보증인 정보 삭제 위해 새보증인상태를 읽음
    event = pd.read_pickle(files[3])
    raw_data = pd.read_pickle(files[4])
    
# 키리스트
keys = pd.read_excel(join(path_excel, fn_keys), dtype=str)
    # 새채무자키 병합
keys = keys.merge(debt[["채무자키","관리자비고"]], how='left')

### A.차주 일반정보 시트

In [ ]:
#######################################################
차주기본정보 = debt[["채무자키","개인법인","성별","연령"]]
#######################################################
차주일반정보 = keys.merge(차주기본정보, on='채무자키', how='left')
차주일반정보 = 차주일반정보.merge(debt[["채무자키","초본주소"]], on='채무자키', how='left')

# 지역 정보
차주일반정보["권역"] = 차주일반정보["초본주소"].str.split().str[0]
차주일반정보["권역"] = 차주일반정보["권역"].str.split('특별').str[0]
차주일반정보["권역"] = 차주일반정보["권역"].str.split('광역').str[0]
차주일반정보["권역"] = 차주일반정보["권역"].replace({'전라남도':'전남','경상북도':'경북', '경기도':'경기', '충청남도':'충남', '충청북도':'충북'})
# 이상없는지 체크
차주일반정보["권역"].unique()

### 채권정보

In [ ]:
# 채권 기본정보
채권정보 = keys.merge(raw_data[["계좌키", "계좌번호", "계정과목","은행대출원금","최초대출일","대출만기일","연체이자율","최초연체일","현재원금","현재법비용","OPB","현재미수이자","현재연체이자","현재합계","시효완성일","원채권사","매입일","시효사유"]], on='계좌키', how='left')
# 금액 합계 맞는지 체크 : 25년 4월까지 자산정리 파일은 현재합계가 잘못됨(이자 차감 문제)
print(all(채권정보[["현재원금","현재법비용"]].sum(axis=1) == 채권정보.OPB))
print(all(채권정보[["OPB","현재미수이자", "현재연체이자"]].sum(axis=1) == 채권정보.OPB))

In [ ]:
# # 수정
# 채권정보["현재합계"] = 채권정보["현재합계"] = 채권정보[["OPB","현재미수이자", "현재연체이자"]].sum(axis=1) # +로 연결하는 경우 na값이 있으면 na가 되어버림

0     19,974,821
1     14,515,617
2     70,714,896
3     12,993,678
4     27,614,654
         ...    
861   20,410,047
862    1,929,992
863    1,541,848
864            0
865   67,845,077
Length: 866, dtype: float64

### 법조치정보
- 새채무자키 기준으로
- event에 사건번호 없는 것은 제거되어 있음. 중복사건은 있음(채무자키가 다르거나, 승계 등으로 다건 가능)

In [ ]:
# 새채무자키 입히기
event = event.merge(debt[["채무자키","관리자비고"]], how='left')

# 청구금액은 법조치 등록시의 전산현재합계금, 원금은 소송목적가액이나 없는 경우 많음
# 종료일(결정일, 확정일x): 확정일이나 종국결과일(전산이 자동 입력)은 없는 경우 많음
법조치정보 = event[event["관리자비고"].isin(keys["관리자비고"])][["채무자키","채무자키","법조치구분","청구금액", "관할법원","사건번호","종료일"]]

### 보증인정보

In [64]:
매각불가상태 = ['종결','종료','불가','면책']
보증인정보 = grt[grt.채무자키.isin(keys.채무자키) & ~grt.새보증인상태.isin(매각불가상태)][["채무자키","계좌키","채무자관계","계좌번호","보증구분","보증한도액","새보증인상태","보증일"]] # 보증한도 없으면 보증잔액이 곧 한도

### 회수정보
- 월별 회수내역 붙이기

In [ ]:
입금 = pd.read_excel(functions.키워드로파일명찾기(join(path_excel), "입금조회새창_.*월별회수"), dtype = dtype)

# 입금구분 정리 및 복사
입금 = 입금[~입금.입금구분.str.contains("부채|증명|해지|환매|매각|기타|환급|법착")].copy() # 법원환급금 제외
try :
    입금["입금월"] = 입금.입금일.apply(lambda x:x[:7])
except : # 임시저장하면 칼럼명이 입금적용일
    입금["입금월"] = 입금.입금적용일.apply(lambda x:x[:7])

# 피벗테이블
월별입금 = 입금[["계좌키", "입금월", "입금합계"]].pivot_table(index="계좌키", columns='입금월', values='입금합계', aggfunc='sum')
# merge
회수정보 = keys.merge(월별입금, on='계좌키', how='left')
회수정보 = 회수정보.fillna(0)

### 모든 시트에 차주 기본정보 붙이기

In [ ]:
# 모든 시트에 차주 기본정보 붙이기
채권정보 = 채권정보.merge(차주기본정보, on='채무자키', how='left') # 중복없는 debt를 써야함
법조치정보 = 법조치정보.merge(차주기본정보, on='채무자키', how='left')
보증인정보 = 보증인정보.merge(차주기본정보, on='채무자키', how='left')
회수정보 = 회수정보.merge(차주기본정보, on='채무자키', how='left')

### 엑셀 저장
- 차주 정보는 당사 채무자키, 새채무자키 두 가지로 분리하여 저장

In [89]:
# 차주일반정보 중복제거 및 채권잔액 정보 붙이기
    # 기준1) 채무자키
기준열 = '채무자키'
        # 채권잔액 계산
채권잔액_채무자키 = 채권정보.groupby(기준열)[["현재원금", "현재법비용", "OPB", "현재미수이자", "현재연체이자", "현재합계"]].sum().reset_index()
        # 키 중복제거
차주일반정보_채무자키 = 차주일반정보.drop_duplicates(subset=기준열)
        # 병합
차주일반정보_채무자키 = 차주일반정보_채무자키.merge(채권잔액_채무자키, on=기준열, how="left")

    # 기준2) 새채무자키
기준열 = '관리자비고'
        # 채권잔액 계산
채권잔액_새채무자키 = 채권정보.groupby(기준열)[["현재원금", "현재법비용", "OPB", "현재미수이자", "현재연체이자", "현재합계"]].sum().reset_index()
        # 키 중복제거
차주일반정보_새채무자키 = 차주일반정보.drop_duplicates(subset=기준열)
        # 병합
차주일반정보_새채무자키 = 차주일반정보_새채무자키.merge(채권잔액_새채무자키, on=기준열, how="left")

# 체크
print(f'중복제거전:{len(차주일반정보)}, 채무자키로 중복제거후:{len(차주일반정보_채무자키)}, 새채무자키로 중복제거후{len(차주일반정보_새채무자키)} ')
print(f'총합게 서로 같은지 : {차주일반정보_채무자키.현재합계.sum() == 차주일반정보_새채무자키.현재합계.sum()}')

중복제거전:866, 채무자키로 중복제거후:866, 새채무자키로 중복제거후722 
총합게 서로 같은지 : True


In [ ]:
with pd.ExcelWriter(join(path_excel, "DataDisk.xlsx"), engine='openpyxl', mode='w') as writer : 
    차주일반정보_채무자키.to_excel(writer, sheet_name='차주일반정보_채무자키', index=False)
    차주일반정보_새채무자키.to_excel(writer, sheet_name='차주일반정보_새채무자키', index=False)
    채권정보.to_excel(writer, sheet_name='채권정보', index=False)
    법조치정보.to_excel(writer, sheet_name='법조치정보', index=False)
    보증인정보.to_excel(writer, sheet_name='보증인정보', index=False)
    회수정보.to_excel(writer, sheet_name='회수정보', index=False)